# <center> CIFAR-10
by Hyr1sky_He

~~老朋友了，上次拿ResNet写的，这次试试MLP~~

## 1. 数据处理

### 1.1 读入数据

In [1]:
import numpy as np
import pandas as pd
import time
import torch
import torchvision
import matplotlib.pyplot as plt
%matplotlib inline

# Constants
IMAGE_SIZE = 32
NUM_CLASSES = 10
COLOR_CHANNELS = 3

In [3]:
cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, '\nCUDA:', cuda)

Using PyTorch version: 2.0.1 
CUDA: True


In [7]:
# Load the dataset
cifar10 = torchvision.datasets.CIFAR10(root='../Assignment1_dataset/', train = True, download = False)
cifar10_test = torchvision.datasets.CIFAR10(root='../Assignment1_dataset/', train = False, download = False)

### 1.2 观察数据

把Batch_1和meta先读进来，看看Label

In [29]:
data_batch_1 = pd.read_pickle('../Assignment1_dataset/cifar-10-batches-py/data_batch_1')
meta = pd.read_pickle('../Assignment1_dataset/cifar-10-batches-py/batches.meta')
test_batch = pd.read_pickle('../Assignment1_dataset/cifar-10-batches-py/test_batch')

print(data_batch_1.keys())
print(meta.keys())

dict_keys(['batch_label', 'labels', 'data', 'filenames'])
dict_keys(['num_cases_per_batch', 'label_names', 'num_vis'])


meta包含了数据集的各种label和信息，再细看一下，顺便把batch_1独热编码了。

In [30]:
one_hot_labels = set(data_batch_1['labels'])
print(one_hot_labels)
label_names = meta['label_names']
print(label_names, '\n')
label_dict = dict(zip(one_hot_labels, label_names))
print('字典')
for i in range(10):
    print(i, ':', label_dict[i])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

字典
0 : airplane
1 : automobile
2 : bird
3 : cat
4 : deer
5 : dog
6 : frog
7 : horse
8 : ship
9 : truck


In [34]:
train_data_1 = list(data_batch_1['labels'])
test_data = list(test_batch['labels'])
print('train_data:', len(train_data_1))
print('test_data:', len(test_data))

train_data: 10000
test_data: 10000
